In [3]:
import tkinter as tk
from tkinter import messagebox, ttk
import re
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, _tree
from sklearn.model_selection import train_test_split
import pyttsx3
import csv

# Load data
training = pd.read_csv('Training.csv')
testing = pd.read_csv('Testing.csv')
cols = training.columns[:-1]
x = training[cols]
y = training['prognosis']

# Reduced data
reduced_data = training.groupby('prognosis').max()

# Mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx = testing[cols]
testy = le.transform(testing['prognosis'])

# Train DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(x_train, y_train)

# Initialize dictionaries
severityDictionary = {}
description_list = {}
precautionDictionary = {}

# Create symptoms dictionary
symptoms_dict = {symptom: index for index, symptom in enumerate(x.columns)}

def readn(nstr):
    engine = pyttsx3.init()
    engine.setProperty('voice', "english+f5")
    engine.setProperty('rate', 130)
    engine.say(nstr)
    engine.runAndWait()
    engine.stop()

def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if len(row) >= 2:
                description_list[row[0]] = row[1]

def getSeverityDict():
    global severityDictionary
    with open('symptom_severity.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if len(row) >= 2:
                severityDictionary[row[0]] = int(row[1])

def getPrecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if len(row) >= 5:
                precautionDictionary[row[0]] = [row[1], row[2], row[3], row[4]]

getSeverityDict()
getDescription()
getPrecautionDict()

def calc_condition(exp, days):
    total_severity = sum(severityDictionary[item] for item in exp)
    if (total_severity * days) / (len(exp) + 1) > 13:
        return "You should take the consultation from a doctor."
    else:
        return "It might not be that bad but you should take precautions."

def sec_predict(symptoms_exp):
    df = pd.read_csv('Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier().fit(X_train, y_train)

    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
        input_vector[symptoms_dict[item]] = 1

    input_df = pd.DataFrame([input_vector], columns=X.columns)
    return rf_clf.predict(input_df)

def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return [d.strip() for d in disease]

def check_pattern(dis_list, inp):
    inp = inp.replace(' ', '_')
    pattern = re.compile(inp)
    pred_list = [item for item in dis_list if pattern.search(item)]
    return (1, pred_list) if pred_list else (0, [])

def tree_to_code(tree, feature_names, disease_input, num_days):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = feature_names
    symptoms_present = []

    def recurse(node, depth):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            val = 1 if name == disease_input else 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])
            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]

            symptoms_exp = []
            for syms in symptoms_given:
                if messagebox.askyesno("Symptom Check", f"Are you experiencing {syms}?"):
                    symptoms_exp.append(syms)

            second_prediction = sec_predict(symptoms_exp)
            condition = calc_condition(symptoms_exp, num_days)

            if present_disease[0] == second_prediction[0]:
                diagnosis = present_disease[0]
            else:
                diagnosis = f"{present_disease[0]} or {second_prediction[0]}"

            description = description_list[present_disease[0]]
            precautions = precautionDictionary[present_disease[0]]

            result = f"You may have {diagnosis}\n\n{description}\n\n{condition}\n\nTake the following measures:\n"
            for i, precaution in enumerate(precautions):
                result += f"{i + 1}. {precaution}\n"

            readn(result)
            messagebox.showinfo("Diagnosis Result", result)

    recurse(0, 1)

# GUI Implementation
class HealthCareChatbot:
    def __init__(self, root):
        self.root = root
        self.root.title("HealthCare Chatbot")

        # Styling
        self.root.configure(bg='#FFFFFF')
        self.root.geometry('600x400')

        self.label_name = tk.Label(root, text="HealthCare Chatbot", font=("Arial", 24, 'bold'), bg='#FFFFFF')
        self.label_name.pack(pady=10)

        self.label_name = tk.Label(root, text="What is your name?", font=("Arial", 14), bg='#FFFFFF')
        self.label_name.pack(pady=5)

        self.name_entry = tk.Entry(root, width=50, font=("Arial", 14))
        self.name_entry.pack(pady=5)

        self.submit_name_button = tk.Button(root, text="Submit", command=self.submit_name, font=("Arial", 14), bg='#3b8ea5', fg='#FFFFFF')
        self.submit_name_button.pack(pady=10)

    def submit_name(self):
        self.name = self.name_entry.get().strip()
        if self.name:
            readn(f"Hello, {self.name}")
            self.ask_symptom()
        else:
            messagebox.showerror("Input Error", "Name cannot be empty.")

    def ask_symptom(self):
        for widget in self.root.winfo_children():
            widget.destroy()

        self.label = tk.Label(self.root, text="Enter the symptom you are experiencing:", font=("Arial", 14), bg='#FFFFFF')
        self.label.pack(pady=10)

        self.symptom_entry = tk.Entry(self.root, width=50, font=("Arial", 14))
        self.symptom_entry.pack(pady=5)

        self.label_days = tk.Label(self.root, text="For how many days?", font=("Arial", 14), bg='#FFFFFF')
        self.label_days.pack(pady=5)

        self.days_entry = tk.Entry(self.root, width=10, font=("Arial", 14))
        self.days_entry.pack(pady=5)

        self.submit_button = tk.Button(self.root, text="Submit", command=self.submit, font=("Arial", 14), bg='#3b8ea5', fg='#FFFFFF')
        self.submit_button.pack(pady=20)

    def submit(self):
        disease_input = self.symptom_entry.get().strip()
        try:
            num_days = int(self.days_entry.get().strip())
        except ValueError:
            messagebox.showerror("Invalid input", "Please enter a valid number of days.")
            return

        if disease_input:
            check, matched_symptoms = check_pattern(cols, disease_input)
            if check:
                if len(matched_symptoms) > 1:
                    symptom_choice = tk.Toplevel(self.root)
                    symptom_choice.title("Choose Symptom")
                    symptom_choice.geometry('400x300')
                    symptom_choice.configure(bg='#FFFFFF')

                    label = tk.Label(symptom_choice, text="Please choose the symptom you meant:", font=("Arial", 14), bg='#FFFFFF')
                    label.pack(pady=10)

                    var = tk.StringVar(value=matched_symptoms[0])

                    for symptom in matched_symptoms:
                        tk.Radiobutton(symptom_choice, text=symptom, variable=var, value=symptom, font=("Arial", 12), bg='#FFFFFF').pack(anchor=tk.W)

                    def choose():
                        chosen_symptom = var.get()
                        symptom_choice.destroy()
                        tree_to_code(clf, cols, chosen_symptom, num_days)

                    choose_button = tk.Button(symptom_choice, text="Choose", command=choose, font=("Arial", 14), bg='#3b8ea5', fg='#FFFFFF')
                    choose_button.pack(pady=20)

                else:
                    tree_to_code(clf, cols, matched_symptoms[0], num_days)
            else:
                messagebox.showerror("Invalid Symptom", "Please enter a valid symptom.")
        else:
            messagebox.showerror("Input Error", "Symptom input cannot be empty.")

if __name__ == "__main__":
    root = tk.Tk()
    app = HealthCareChatbot(root)
    root.mainloop()

